In [1]:
import pandas as pd
import numpy as np
import os
import csv

In [2]:
#Event='AscendEx'
Event='Upbit'
raw_path='../Dataset/'+Event+'/all_data_token/'
address_path='../Dataset/'+Event+'/dataset/'

In [3]:
with open('../Code_Results/Tx_Velocity_Results/'+Event+'_Tx_Velocity.csv','w') as f:

    csv_write = csv.writer(f)
    csv_head = ["address","LifeTime","Tx_Num","Tx_Velocity"]
    csv_write.writerow(csv_head)

output = pd.read_csv('../Code_Results/Tx_Velocity_Results/'+Event+'_Tx_Velocity.csv')

In [4]:
#Step1：筛选出之发生了ETH交易的洗钱账户（source+layering）
#方法2：all_erc20_address和all_normal_address的交集，再用all_normal_address减掉交集
all_address=pd.read_csv(address_path+"all-address.csv")
all_erc20_address=pd.read_csv(address_path+"all-erc20-address.csv")
all_normal_address=pd.read_csv(address_path+"all-normal-address.csv")
address=all_address['address']
erc20_address=all_erc20_address['address']
normal_address=all_normal_address['address']
ETH_address=set(address)-set(erc20_address)
temp=set(erc20_address)&set(normal_address)
ETH_address=list(set(normal_address)-temp)
#再筛选出来label是heist的账户
all_ETH_address=all_normal_address[all_normal_address['address'].isin(ETH_address)]
# ETH_heist_address=list(all_ETH_address.loc[(all_ETH_address['label']=='heist')&(all_ETH_address['name_tag']!='ml_transit_9')]['address'])
ETH_heist_address=list(all_ETH_address.loc[(all_ETH_address['label']=='heist')]['address'])

In [5]:
output['address']=ETH_heist_address

In [6]:
#Step2：筛选出ETH账户的交易
all_normal_tx=pd.read_csv(address_path+"all-normal-tx.csv")
ETH_tx=all_normal_tx[all_normal_tx['from'].isin(list(ETH_heist_address)) | all_normal_tx['to'].isin(list(ETH_heist_address))]

In [16]:
#ETH_heist_address=['0xe7ef8592e64df04891e1e3bbf3013970b5a54f43']

In [25]:
#Step3：统计每个账户的交易速度
for i in range(0,len(ETH_heist_address)):
    print('%d/%d'%(i,len(ETH_heist_address)),end="\r")
    #record=ETH_tx.loc[ETH_tx["isError"]!=1]
    record=ETH_tx
    Ac=ETH_heist_address[i]
    Ac_Tx=record.loc[(record['from']==Ac)|(record['to']==Ac)]
    Tx_Num=len(Ac_Tx)
    #统计交易的时间范围
    timestamp=Ac_Tx['timeStamp']
    min_T=min(timestamp)
    max_T=max(timestamp)
    Del_T=max_T-min_T
    if Del_T !=0:
        Ac_Velocity=Tx_Num/Del_T*60*60#转为小时
    else:
        Ac_Velocity=0
    output["LifeTime"][i]=Del_T
    output["Tx_Num"][i]=Tx_Num
    output["Tx_Velocity"][i]=Ac_Velocity
    

11/9968

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


5724/9968

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [20]:
output=output.loc[output['Tx_Velocity']!=0]

In [26]:
output

,address,LifeTime,Tx_Num,Tx_Velocity
0,0xe7ef8592e64df04891e1e3bbf3013970b5a54f43,142920101,17,0.000428211
1,0x7916ed1b5accf672e277fc06b1538a21a30bfbee,178940570,70,0.00140829
2,0x170dedd533c37facd60ee18b51d7bcde274a0d1f,172966195,70,0.00145693
3,0x16ffc8a17b60398e4fcdc8c111e56373a60d8db1,126864707,171,0.00485241
4,0x6c134879dab2103aa5b55e90a16eaf96d0f31a2c,170261896,43,0.000909188
...,...,...,...,...
9957,0x4b0a5cef6a7b3289fc32ac1699a83edf45b22d0f,3548,3,3.04397
9958,0xa071a38b718502934bf99f2386fe3e3a1c4799fb,11451081,2,0.000628762
9960,0xf387e4186b93f238e3830837bbb39de0431bc966,5794328,22,0.0136685
9962,0xcc41b70d6c981d72ac78d3841f6d82e57f3190be,5257741,2,0.00136941


In [27]:
np.mean(output['Tx_Velocity'])

196.38995011222775

In [28]:
max(output['Tx_Velocity'])

18000.0

In [30]:
output.loc[output['Tx_Velocity']==18000.0]

,address,LifeTime,Tx_Num,Tx_Velocity
3792,0xb45233774ec819e92231cc786ec28136e1017727,1,5,18000


In [29]:
min(output['Tx_Velocity'])

8.77569377892612e-05

In [35]:
np.median(output['Tx_Velocity'])

18.604664965279838